<a href="https://colab.research.google.com/github/basangoudapatil/Task---7-8/blob/main/Assignment_10(KNN_Classifier).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You are provided with a dataset from USA Forensic Science Service
which has description of 6 types of glass; defined in terms of their
oxide content (i.e. Na, Fe, K, etc). Your task is to use K-Nearest
Neighbor (KNN) classifier to classify the glasses.

The original dataset is available at
(https://archive.ics.uci.edu/ml/datasets/glass+identification). 

For
detailed description on the attributes of the dataset,
please refer to the original link of the dataset in the UCI ML
repository.

Perform exploratory data analysis on the dataset using Python Pandas,
including dropping irrelevant fields for predicted values, and
standardization of each attribute.
Following data cleaning, two Scikit-Learn KNN models should be created
for two different distance metrics: Square Euclidean and Manhattan
distance. The performance of the two models using different distance
metrics should be compared in terms of accuracy to the test data and
Scikit-Learn Classification Report.

In [1]:
import pandas as pd
import numpy as np

In [12]:
train = pd.read_csv('/content/trainKNN.txt', header = None)
train.columns = ['ID','RI','Na','Mg','Al','Si','K','Ca','Ba','Fe', 'Type of Glass']
test = pd.read_csv('/content/testKNN.txt', header = None)
test.columns = ['ID','RI','Na','Mg','Al','Si','K','Ca','Ba','Fe', 'Type of Glass']

Train Dataset

In [18]:
train = train.drop('ID', axis=1)
print(train.head())

        RI     Na    Mg    Al     Si     K    Ca   Ba   Fe  Type of Glass
0  1.52101  13.64  4.49  1.10  71.78  0.06  8.75  0.0  0.0              1
1  1.51761  13.89  3.60  1.36  72.73  0.48  7.83  0.0  0.0              1
2  1.51618  13.53  3.55  1.54  72.99  0.39  7.78  0.0  0.0              1
3  1.51766  13.21  3.69  1.29  72.61  0.57  8.22  0.0  0.0              1
4  1.51742  13.27  3.62  1.24  73.08  0.55  8.07  0.0  0.0              1


In [44]:
train.isnull().sum()

RI               0
Na               0
Mg               0
Al               0
Si               0
K                0
Ca               0
Ba               0
Fe               0
Type of Glass    0
dtype: int64

Test Dataset

In [19]:
test = test.drop('ID', axis=1)
print(test.head())

        RI     Na    Mg    Al     Si     K     Ca   Ba    Fe  Type of Glass
0  1.52152  13.05  3.65  0.87  72.32  0.19   9.85  0.0  0.17              1
1  1.52152  13.12  3.58  0.90  72.20  0.23   9.82  0.0  0.16              1
2  1.52300  13.31  3.58  0.82  71.99  0.12  10.17  0.0  0.03              1
3  1.51709  13.00  3.47  1.79  72.72  0.66   8.18  0.0  0.00              2
4  1.51660  12.99  3.18  1.23  72.97  0.58   8.81  0.0  0.24              2


In [45]:
test.isnull().sum()

RI               0
Na               0
Mg               0
Al               0
Si               0
K                0
Ca               0
Ba               0
Fe               0
Type of Glass    0
dtype: int64

In [23]:
train['Type of Glass'].value_counts()

2    73
1    67
7    26
3    14
5    10
6     6
Name: Type of Glass, dtype: int64

In [21]:
test['Type of Glass'].value_counts()

1    3
2    3
3    3
5    3
6    3
7    3
Name: Type of Glass, dtype: int64

Understanding the Data

In [24]:
train.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of Glass
count,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000,196.000000
mean,1.518295,13.375204,2.758980,1.454337,72.635408,0.519388,8.910714,0.164235,0.050255,2.668367
std,0.003055,0.783145,1.392641,0.491688,0.763578,0.672703,1.421490,0.485198,0.086359,2.062416
min,1.511310,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516458,12.877500,2.362500,1.190000,72.317500,0.140000,8.220000,0.000000,0.000000,1.000000
50%,1.517630,13.280000,3.480000,1.360000,72.810000,0.560000,8.575000,0.000000,0.000000,2.000000
75%,1.518985,13.792500,3.610000,1.622500,73.080000,0.610000,9.092500,0.000000,0.090000,3.000000
max,1.533930,15.790000,4.490000,3.500000,75.180000,6.210000,16.190000,3.150000,0.340000,7.000000


In [25]:
test.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of Glass
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,1.519131,13.763333,1.873889,1.342222,72.820000,0.253889,9.460556,0.292778,0.130556,4.000000
std,0.002791,1.083491,1.749753,0.581312,0.892004,0.265133,1.380432,0.617422,0.164798,2.222876
min,1.511150,12.850000,0.000000,0.340000,71.360000,0.000000,6.650000,0.000000,0.000000,1.000000
25%,1.517412,13.012500,0.000000,0.877500,72.212500,0.000000,8.635000,0.000000,0.000000,2.000000
50%,1.519515,13.355000,2.395000,1.320000,72.685000,0.175000,9.065000,0.000000,0.015000,4.000000
75%,1.521055,14.220000,3.570000,1.902500,73.382500,0.502500,10.090000,0.112500,0.240000,6.000000
max,1.523000,17.380000,3.780000,2.170000,75.410000,0.760000,12.500000,1.670000,0.510000,7.000000


In [30]:
#Standardization of Train and Test dataset
def standardize(data):
  for col in data.columns:
    if col != 'Type of Glass':
      data[col]=(data[col] - data[col].mean())/data[col].std()
  return data 

In [32]:
train = standardize(train)
test = standardize(test)
train.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of Glass
0,0.888537,0.338119,1.242977,-0.720654,-1.120263,-0.682898,-0.113060,-0.33849,-0.581932,1
1,-0.224221,0.657345,0.603903,-0.191863,0.123880,-0.058551,-0.760269,-0.33849,-0.581932,1
2,-0.692234,0.197659,0.568000,0.174223,0.464382,-0.192340,-0.795443,-0.33849,-0.581932,1
3,-0.207857,-0.210950,0.668529,-0.334230,-0.033275,0.075237,-0.485909,-0.33849,-0.581932,1
4,-0.286405,-0.134335,0.618265,-0.435920,0.582248,0.045506,-0.591432,-0.33849,-0.581932,1


In [33]:
test.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of Glass
0,0.856076,-0.658366,1.015064,-0.812338,-0.560536,-0.240969,0.282118,-0.474194,0.239350,1
1,0.856076,-0.593760,0.975058,-0.760731,-0.695064,-0.090101,0.260385,-0.474194,0.178670,1
2,1.386446,-0.418401,0.975058,-0.898350,-0.930489,-0.504987,0.513929,-0.474194,-0.610173,1
3,-0.731448,-0.704513,0.912192,0.770288,-0.112107,1.531723,-0.927648,-0.474194,-0.792214,2
4,-0.907043,-0.713742,0.746454,-0.193050,0.168161,1.229989,-0.471269,-0.474194,0.664111,2


In [38]:
#Preparing data for Analysis
X_train = train.drop(['Type of Glass'], axis = 1).values
y_train = train['Type of Glass'].values

X_test = test.drop(['Type of Glass'], axis = 1).values
y_test = test['Type of Glass'].values

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(196, 9)
(196,)
(18, 9)
(18,)


In [53]:
#K-NN Algorithm for the model
from scipy.spatial import distance

#Giving K value as Odd for first instance
k = 5

from sklearn.neighbors import KNeighborsClassifier

Euclid_model = KNeighborsClassifier(n_neighbors= k, metric= distance.sqeuclidean)
Manhattan_model = KNeighborsClassifier(n_neighbors= k, metric = distance.cityblock)

In [43]:
#Training the Euclidean Model for Train datatset
#Predict the values for the test Dataset
Euclid_model.fit(X_train, y_train)

Euclid_predictions = Euclid_model.predict(X_test)

df = pd.DataFrame({'Actual': y_test, 'Predicted': Euclid_predictions})
print(df)

    Actual  Predicted
0        1          1
1        1          1
2        1          1
3        2          2
4        2          1
5        2          2
6        3          2
7        3          2
8        3          1
9        5          5
10       5          2
11       5          5
12       6          6
13       6          2
14       6          7
15       7          7
16       7          7
17       7          7


In [47]:
#Training the Manhattan Model for Train datatset
#Predict the values for the test Dataset
Manhattan_model.fit(X_train, y_train)

Manhattan_predictions = Manhattan_model.predict(X_test)

df1 = pd.DataFrame({'ACtual': y_test, 'Predicted': Manhattan_predictions})
print(df1)

    ACtual  Predicted
0        1          1
1        1          1
2        1          1
3        2          2
4        2          1
5        2          1
6        3          2
7        3          2
8        3          1
9        5          5
10       5          2
11       5          5
12       6          6
13       6          6
14       6          7
15       7          7
16       7          7
17       7          7


In [48]:
df3 = pd.DataFrame({'Actual': y_test, 'Euclid': Euclid_predictions , 'Manhattan': Manhattan_predictions})
print(df3)

    Actual  Euclid  Manhattan
0        1       1          1
1        1       1          1
2        1       1          1
3        2       2          2
4        2       1          1
5        2       2          1
6        3       2          2
7        3       2          2
8        3       1          1
9        5       5          5
10       5       2          2
11       5       5          5
12       6       6          6
13       6       2          6
14       6       7          7
15       7       7          7
16       7       7          7
17       7       7          7


In [49]:
#Evaluation Metrics
from sklearn import metrics

print('Train Set Accuracy - Euclidean-Model:', metrics.accuracy_score(y_train, Euclid_model.predict(X_train)))
print('Test Set Accuracy - Euclidean-Model:', metrics.accuracy_score(y_test, Euclid_predictions))

Train Set Accuracy - Euclidean-Model: 0.8010204081632653
Test Set Accuracy - Euclidean-Model: 0.6111111111111112


In [50]:
#Evaluation Metrics
from sklearn import metrics

print('Train Set Accuracy - Manhattan-Model:', metrics.accuracy_score(y_train, Manhattan_model.predict(X_train)))
print('Test Set Accuracy - Manhattan-Model:', metrics.accuracy_score(y_test, Manhattan_predictions))

Train Set Accuracy - Manhattan-Model: 0.8010204081632653
Test Set Accuracy - Manhattan-Model: 0.6111111111111112


In [72]:
#For k=8 
#Even Value for Second Instance
k=8

Euclid_model = KNeighborsClassifier(n_neighbors= k, metric= distance.sqeuclidean)
Manhattan_model = KNeighborsClassifier(n_neighbors= k, metric = distance.cityblock)

In [73]:
#Training the Euclidean Model for Train datatset
#Predict the values for the test Dataset
Euclid_model.fit(X_train, y_train)

Euclid_predictions = Euclid_model.predict(X_test)

df = pd.DataFrame({'Actual': y_test, 'Predicted': Euclid_predictions})
print(df)

    Actual  Predicted
0        1          1
1        1          1
2        1          1
3        2          2
4        2          1
5        2          2
6        3          2
7        3          2
8        3          1
9        5          5
10       5          2
11       5          5
12       6          6
13       6          2
14       6          7
15       7          7
16       7          7
17       7          7


In [74]:
#Training the Manhattan Model for Train datatset
#Predict the values for the test Dataset
Manhattan_model.fit(X_train, y_train)

Manhattan_predictions = Manhattan_model.predict(X_test)

df1 = pd.DataFrame({'ACtual': y_test, 'Predicted': Manhattan_predictions})
print(df1)

    ACtual  Predicted
0        1          1
1        1          1
2        1          1
3        2          2
4        2          1
5        2          1
6        3          2
7        3          2
8        3          1
9        5          5
10       5          2
11       5          5
12       6          6
13       6          6
14       6          6
15       7          7
16       7          7
17       7          7


In [75]:
df4 = pd.DataFrame({'Actual': y_test, 'Euclid': Euclid_predictions , 'Manhattan': Manhattan_predictions})
print(df4)

    Actual  Euclid  Manhattan
0        1       1          1
1        1       1          1
2        1       1          1
3        2       2          2
4        2       1          1
5        2       2          1
6        3       2          2
7        3       2          2
8        3       1          1
9        5       5          5
10       5       2          2
11       5       5          5
12       6       6          6
13       6       2          6
14       6       7          6
15       7       7          7
16       7       7          7
17       7       7          7


In [76]:
#Evaluation Metrics
from sklearn import metrics

print('Train Set Accuracy - Euclidean-Model:', metrics.accuracy_score(y_train, Euclid_model.predict(X_train)))
print('Test Set Accuracy - Euclidean-Model:', metrics.accuracy_score(y_test, Euclid_predictions))

#Evaluation Metrics
from sklearn import metrics

print('Train Set Accuracy - Manhattan-Model:', metrics.accuracy_score(y_train, Manhattan_model.predict(X_train)))
print('Test Set Accuracy - Manhattan-Model:', metrics.accuracy_score(y_test, Manhattan_predictions))

Train Set Accuracy - Euclidean-Model: 0.7551020408163265
Test Set Accuracy - Euclidean-Model: 0.6111111111111112
Train Set Accuracy - Manhattan-Model: 0.7602040816326531
Test Set Accuracy - Manhattan-Model: 0.6666666666666666


In [63]:
#Evaluation Performance for k=5
from sklearn.metrics import classification_report
Manhattan_count = len(df3.loc[df3['Manhattan'] == df3['Actual']])
Euclid_count = len(df3.loc[df3['Euclid'] == df3['Actual']])

In [67]:
print('Manhattan Accuracy: {}%'.format(round(100*Manhattan_count/len(df3), 2)))
print(classification_report(y_test, Manhattan_predictions, target_names=df3['Actual'].astype(str).unique()))

Manhattan Accuracy: 61.11%
              precision    recall  f1-score   support

           1       0.50      1.00      0.67         3
           2       0.20      0.33      0.25         3
           3       0.00      0.00      0.00         3
           5       1.00      0.33      0.50         3
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         3

    accuracy                           0.61        18
   macro avg       0.62      0.61      0.57        18
weighted avg       0.62      0.61      0.57        18



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [68]:
print('Square Euclidean Accuracy: {}%'.format(round(100*Euclid_count/len(df3), 2)))
print(classification_report(y_test, Euclid_predictions, target_names=df3['Actual'].astype(str).unique()))

Square Euclidean Accuracy: 61.11%
              precision    recall  f1-score   support

           1       0.60      1.00      0.75         3
           2       0.33      0.67      0.44         3
           3       0.00      0.00      0.00         3
           5       1.00      0.67      0.80         3
           6       1.00      0.33      0.50         3
           7       0.75      1.00      0.86         3

    accuracy                           0.61        18
   macro avg       0.61      0.61      0.56        18
weighted avg       0.61      0.61      0.56        18



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
#Evaluation Performance for k=8
from sklearn.metrics import classification_report
Manhattan_count = len(df4.loc[df4['Manhattan'] == df4['Actual']])
Euclid_count = len(df4.loc[df4['Euclid'] == df4['Actual']])

In [78]:
print('Manhattan Accuracy: {}%'.format(round(100*Manhattan_count/len(df4), 2)))
print(classification_report(y_test, Manhattan_predictions, target_names=df4['Actual'].astype(str).unique()))

Manhattan Accuracy: 66.67%
              precision    recall  f1-score   support

           1       0.50      1.00      0.67         3
           2       0.25      0.33      0.29         3
           3       0.00      0.00      0.00         3
           5       1.00      0.67      0.80         3
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         3

    accuracy                           0.67        18
   macro avg       0.62      0.67      0.63        18
weighted avg       0.62      0.67      0.63        18



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [79]:
print('Square Euclidean Accuracy: {}%'.format(round(100*Euclid_count/len(df4), 2)))
print(classification_report(y_test, Euclid_predictions, target_names=df4['Actual'].astype(str).unique()))

Square Euclidean Accuracy: 61.11%
              precision    recall  f1-score   support

           1       0.60      1.00      0.75         3
           2       0.33      0.67      0.44         3
           3       0.00      0.00      0.00         3
           5       1.00      0.67      0.80         3
           6       1.00      0.33      0.50         3
           7       0.75      1.00      0.86         3

    accuracy                           0.61        18
   macro avg       0.61      0.61      0.56        18
weighted avg       0.61      0.61      0.56        18



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
